# Experimental new pipeline

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from experiments.pipelines.pipeline_rpy2 import Rpy2Pipeline
import multiprocessing

import pandas as pd
from IPython.display import display, HTML
from pandas import DataFrame

## Load analysis and project

In [5]:
analysis = Analysis.objects.get_or_create(id=1)[0]

DEBUG 2016-09-26 11:48:01,407 utils.py:91 | (0.006) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 1 LIMIT 21; args=(1,)


In [6]:
project = Project.objects.get_or_create(id=1)[0]

DEBUG 2016-09-26 11:48:04,423 utils.py:91 | (0.000) SELECT `projects_project`.`id`, `projects_project`.`title`, `projects_project`.`user_owner_id`, `projects_project`.`description`, `projects_project`.`created`, `projects_project`.`modified` FROM `projects_project` WHERE `projects_project`.`id` = 1 LIMIT 21; args=(1,)


## Get files etc

In [7]:
def remove_first_two(path):
    tokens = path.split('/')
    new_path = '/'.join(tokens[2:])
    return new_path
    
def strip_project_from_path(path_list):
    processed = []
    for path in path_list:
        if type(path) is tuple:
            first = remove_first_two(path[0])
            second = remove_first_two(path[1])
            new_path = (first, second)
        else:
            new_path = remove_first_two(path)
        processed.append(new_path)
    return processed

In [8]:
from fileupload.models import Sample

experiment = analysis.experiment
samples = Sample.objects.filter(attribute__comparison__experiment = experiment.id).distinct()
pos_samples = Picture.objects.filter(posdata__sample__in=samples).values_list('file', flat=True)
neg_samples = Picture.objects.filter(negdata__sample__in=samples).values_list('file', flat=True)

print strip_project_from_path(pos_samples)
print strip_project_from_path(neg_samples)

DEBUG 2016-09-26 11:48:09,364 utils.py:91 | (0.018) SELECT `experiments_experiment`.`id`, `experiments_experiment`.`title`, `experiments_experiment`.`created`, `experiments_experiment`.`modified` FROM `experiments_experiment` WHERE `experiments_experiment`.`id` = 1 LIMIT 21; args=(1,)
DEBUG 2016-09-26 11:48:09,391 utils.py:91 | (0.016) SELECT `fileupload_picture`.`file` FROM `fileupload_picture` INNER JOIN `fileupload_samplefilegroup` ON ( `fileupload_picture`.`id` = `fileupload_samplefilegroup`.`posdata_id` ) INNER JOIN `fileupload_sample` ON ( `fileupload_samplefilegroup`.`id` = `fileupload_sample`.`samplefile_id` ) WHERE (`fileupload_sample`.`id`) IN (SELECT DISTINCT U0.`id` FROM `fileupload_sample` U0 INNER JOIN `groups_sampleattribute` U1 ON ( U0.`id` = U1.`sample_id` ) INNER JOIN `groups_attribute` U2 ON ( U1.`attribute_id` = U2.`id` ) INNER JOIN `experiments_attributecomparison` U3 ON ( U2.`id` = U3.`attribute_id` ) INNER JOIN `experiments_comparison` U4 ON ( U3.`comparison_id` 

[u'samples/POS/Beer_1_full1.mzXML', u'samples/POS/Beer_1_full2.mzXML', u'samples/POS/Beer_1_full3.mzXML', u'samples/POS/Beer_2_full1.mzXML', u'samples/POS/Beer_2_full2.mzXML', u'samples/POS/Beer_2_full3.mzXML', u'samples/POS/Beer_3_full1.mzXML', u'samples/POS/Beer_3_full2.mzXML', u'samples/POS/Beer_3_full3.mzXML', u'samples/POS/Beer_4_full1.mzXML', u'samples/POS/Beer_4_full2.mzXML', u'samples/POS/Beer_4_full3.mzXML']
[u'samples/NEG/Beer_1_full1.mzXML', u'samples/NEG/Beer_1_full2.mzXML', u'samples/NEG/Beer_1_full3.mzXML', u'samples/NEG/Beer_2_full1.mzXML', u'samples/NEG/Beer_2_full2.mzXML', u'samples/NEG/Beer_2_full3.mzXML', u'samples/NEG/Beer_3_full1.mzXML', u'samples/NEG/Beer_3_full2.mzXML', u'samples/NEG/Beer_3_full3.mzXML', u'samples/NEG/Beer_4_full1.mzXML', u'samples/NEG/Beer_4_full2.mzXML', u'samples/NEG/Beer_4_full3.mzXML']


## Get groups

In [19]:
res = Group.objects.filter(attribute__comparison__experiment = experiment).values_list('name','attribute__name','attribute__sample__name')
data = [row for row in res]

headers = ['factor', 'level', 'sample']
df = DataFrame(data, columns=headers)
display(df)

DEBUG 2016-09-22 13:57:28,987 utils.py:91 | (0.001) SELECT `groups_group`.`name`, `groups_attribute`.`name`, `fileupload_sample`.`name` FROM `groups_group` INNER JOIN `groups_attribute` ON ( `groups_group`.`id` = `groups_attribute`.`group_id` ) INNER JOIN `experiments_attributecomparison` ON ( `groups_attribute`.`id` = `experiments_attributecomparison`.`attribute_id` ) INNER JOIN `experiments_comparison` ON ( `experiments_attributecomparison`.`comparison_id` = `experiments_comparison`.`id` ) LEFT OUTER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) LEFT OUTER JOIN `fileupload_sample` ON ( `groups_sampleattribute`.`sample_id` = `fileupload_sample`.`id` ) WHERE `experiments_comparison`.`experiment_id` = 4; args=(4,)


,factor,level,sample
0,beer_colour,colour_dark,Beer_1_full1.mzXML
1,beer_colour,colour_dark,Beer_1_full2.mzXML
2,beer_colour,colour_dark,Beer_1_full3.mzXML
3,beer_colour,colour_dark,Beer_2_full1.mzXML
4,beer_colour,colour_dark,Beer_2_full2.mzXML
5,beer_colour,colour_dark,Beer_2_full3.mzXML
6,beer_colour,colour_light,Beer_3_full1.mzXML
7,beer_colour,colour_light,Beer_3_full2.mzXML
8,beer_colour,colour_light,Beer_3_full3.mzXML
9,beer_colour,colour_light,Beer_4_full1.mzXML


In [20]:
f = 'beer_colour'
selected = df.loc[df['factor'] == f]
print selected.level.unique()

[u'colour_dark' u'colour_light']


In [21]:
from experiments.pipelines.pipeline_rpy2 import Factor
import os

factors = []
fs = df.factor.unique()
for f_label in fs:
    factor = Factor(f_label)
    factor_df = df.loc[df['factor'] == f_label]
    levels = factor_df.level.unique()
    for level_label in levels:
        level_df = factor_df.loc[factor_df['level'] == level_label]
        samples = level_df[['sample']].values.flatten().tolist()
        
        # strip the extension from samples
        processed = []
        for samp in samples:
            filename, file_extension = os.path.splitext(samp)
            processed.append(filename)
        
        factor.add_level(level_label, processed)
    factors.append(factor)
        
for factor in factors:
    print factor.label
    print factor.levels
    print factor.level_files
    break

beer_colour
[u'colour_dark', u'colour_light']
{u'colour_dark': [u'Beer_1_full1', u'Beer_1_full2', u'Beer_1_full3', u'Beer_2_full1', u'Beer_2_full2', u'Beer_2_full3'], u'colour_light': [u'Beer_3_full1', u'Beer_3_full2', u'Beer_3_full3', u'Beer_4_full1', u'Beer_4_full2', u'Beer_4_full3']}


## Get standard files

In [22]:
standards = CalibrationSample.objects.filter(project=project, attribute__name='standard').values_list('standardFile__data__file', flat=True)
print strip_project_from_path(standards)

DEBUG 2016-09-22 13:57:43,619 utils.py:91 | (0.001) SELECT `fileupload_projfile`.`file` FROM `fileupload_calibrationsample` INNER JOIN `groups_projfileattribute` ON ( `fileupload_calibrationsample`.`id` = `groups_projfileattribute`.`calibrationsample_id` ) INNER JOIN `groups_attribute` ON ( `groups_projfileattribute`.`attribute_id` = `groups_attribute`.`id` ) LEFT OUTER JOIN `fileupload_standardfilegroup` ON ( `fileupload_calibrationsample`.`standardFile_id` = `fileupload_standardfilegroup`.`id` ) LEFT OUTER JOIN `fileupload_projfile` ON ( `fileupload_standardfilegroup`.`data_id` = `fileupload_projfile`.`id` ) WHERE (`fileupload_calibrationsample`.`project_id` = 4 AND `groups_attribute`.`name` = 'standard'); args=(4, 'standard')


[u'calibration_samples/standard/Std1_1_20150422_150810.csv', u'calibration_samples/standard/Std2_1_20150422_150711.csv', u'calibration_samples/standard/Std3_1_20150422_150553.csv']


In [23]:
qc = CalibrationSample.objects.filter(project=project, attribute__name='qc').values_list('standardFile__posdata__file', 'standardFile__negdata__file')
print strip_project_from_path(qc)

DEBUG 2016-09-22 13:57:43,794 utils.py:91 | (0.001) SELECT `fileupload_projfile`.`file`, T7.`file` FROM `fileupload_calibrationsample` INNER JOIN `groups_projfileattribute` ON ( `fileupload_calibrationsample`.`id` = `groups_projfileattribute`.`calibrationsample_id` ) INNER JOIN `groups_attribute` ON ( `groups_projfileattribute`.`attribute_id` = `groups_attribute`.`id` ) LEFT OUTER JOIN `fileupload_standardfilegroup` ON ( `fileupload_calibrationsample`.`standardFile_id` = `fileupload_standardfilegroup`.`id` ) LEFT OUTER JOIN `fileupload_projfile` ON ( `fileupload_standardfilegroup`.`posdata_id` = `fileupload_projfile`.`id` ) LEFT OUTER JOIN `fileupload_projfile` T7 ON ( `fileupload_standardfilegroup`.`negdata_id` = T7.`id` ) WHERE (`fileupload_calibrationsample`.`project_id` = 4 AND `groups_attribute`.`name` = 'qc'); args=(4, 'qc')


[(u'calibration_samples/POS/Beer_PoolB_full_f.mzXML', u'calibration_samples/NEG/Beer_PoolB_full_f.mzXML'), (u'calibration_samples/POS/Beer_PoolB_full_g.mzXML', u'calibration_samples/NEG/Beer_PoolB_full_g.mzXML'), (u'calibration_samples/POS/Beer_PoolB_full_h.mzXML', u'calibration_samples/NEG/Beer_PoolB_full_h.mzXML'), (u'calibration_samples/POS/Beer_PoolB_full_i.mzXML', u'calibration_samples/NEG/Beer_PoolB_full_i.mzXML')]


In [24]:
blank = CalibrationSample.objects.filter(project=project, attribute__name='blank').values_list('standardFile__posdata__file', 'standardFile__negdata__file')
print strip_project_from_path(blank)

DEBUG 2016-09-22 13:57:44,294 utils.py:91 | (0.001) SELECT `fileupload_projfile`.`file`, T7.`file` FROM `fileupload_calibrationsample` INNER JOIN `groups_projfileattribute` ON ( `fileupload_calibrationsample`.`id` = `groups_projfileattribute`.`calibrationsample_id` ) INNER JOIN `groups_attribute` ON ( `groups_projfileattribute`.`attribute_id` = `groups_attribute`.`id` ) LEFT OUTER JOIN `fileupload_standardfilegroup` ON ( `fileupload_calibrationsample`.`standardFile_id` = `fileupload_standardfilegroup`.`id` ) LEFT OUTER JOIN `fileupload_projfile` ON ( `fileupload_standardfilegroup`.`posdata_id` = `fileupload_projfile`.`id` ) LEFT OUTER JOIN `fileupload_projfile` T7 ON ( `fileupload_standardfilegroup`.`negdata_id` = T7.`id` ) WHERE (`fileupload_calibrationsample`.`project_id` = 4 AND `groups_attribute`.`name` = 'blank'); args=(4, 'blank')


[]


## Get comparisons

In [19]:
comparisons = Comparison.objects.filter(experiment=experiment).values_list(
    'id', 'name', 'attribute__group__name', 'attribute__name', 'attributecomparison__group')
print comparisons

data = [list(row) for row in comparisons]
headers = ['id', 'comparison', 'factor', 'level', 'group']
df = pd.DataFrame(data, columns=headers)
display(df)

DEBUG 2016-09-26 14:24:43,751 utils.py:91 | (0.001) SELECT `experiments_comparison`.`id`, `experiments_comparison`.`name`, `groups_group`.`name`, `groups_attribute`.`name`, `experiments_attributecomparison`.`group` FROM `experiments_comparison` LEFT OUTER JOIN `experiments_attributecomparison` ON ( `experiments_comparison`.`id` = `experiments_attributecomparison`.`comparison_id` ) LEFT OUTER JOIN `groups_attribute` ON ( `experiments_attributecomparison`.`attribute_id` = `groups_attribute`.`id` ) LEFT OUTER JOIN `groups_group` ON ( `groups_attribute`.`group_id` = `groups_group`.`id` ) WHERE `experiments_comparison`.`experiment_id` = 1 LIMIT 21; args=(1,)
DEBUG 2016-09-26 14:24:43,753 utils.py:91 | (0.001) SELECT `experiments_comparison`.`id`, `experiments_comparison`.`name`, `groups_group`.`name`, `groups_attribute`.`name`, `experiments_attributecomparison`.`group` FROM `experiments_comparison` LEFT OUTER JOIN `experiments_attributecomparison` ON ( `experiments_comparison`.`id` = `exper

[(1L, u'beer_colour_comparison', u'beer_colour', u'colour_dark', 0L), (1L, u'beer_colour_comparison', u'beer_colour', u'colour_light', 1L), (2L, u'beer_taste_comparison', u'beer_taste', u'taste_awful', 0L), (2L, u'beer_taste_comparison', u'beer_taste', u'taste_delicious', 1L)]


,id,comparison,factor,level,group
0,1,beer_colour_comparison,beer_colour,colour_dark,0
1,1,beer_colour_comparison,beer_colour,colour_light,1
2,2,beer_taste_comparison,beer_taste,taste_awful,0
3,2,beer_taste_comparison,beer_taste,taste_delicious,1


## Get analysis parameters

In [ ]:
databases = Database.objects.filter(params__analysis=analysis).values_list('name', flat=True)
print type(databases)
print type(list(databases))

In [ ]:
params = Parameter.objects.filter(params__analysis=analysis).values_list('name', 'value', 'state')
print params

## Start the pipeline

In [9]:
pipeline = Rpy2Pipeline(analysis, project, False)

INFO 2016-09-26 11:48:20,300 pipeline_rpy2.py:38 | ******************************************
INFO 2016-09-26 11:48:20,301 pipeline_rpy2.py:39 | Setup rpy2 connection
INFO 2016-09-26 11:48:20,302 pipeline_rpy2.py:40 | ******************************************


[1] "Initialising PuMP logging"
[1] "PuMP log level set to DEBUG"
2016-09-26 11:48:24 ERROR:pump.misc.initialisePuMPLogging:Logging at ERROR level
2016-09-26 11:48:24 WARNING:pump.misc.initialisePuMPLogging:Logging at WARNING level
2016-09-26 11:48:24 INFO:pump.misc.initialisePuMPLogging:Logging at INFO level
2016-09-26 11:48:24 DEBUG:pump.misc.initialisePuMPLogging:Logging at DEBUG level
2016-09-26 11:48:24 INFO:PeakML.Isotope.plotSamples:OS type: x86_64-apple-darwin15.5.0
2016-09-26 11:48:24 INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx512m
2016-09-26 11:48:24 INFO:PeakML.Isotope.plotSamples:Using jar file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2016-09-26 11:48:24 INFO:PeakML.Isotope.plotSamples:Using SWT file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/swt-4.5-cocoa-macosx-x86_64.jar
[1] "Initialising PiMP logging"
[1]

DEBUG 2016-09-26 11:48:25,243 utils.py:91 | (0.001) SELECT `fileupload_picture`.`file` FROM `fileupload_picture` INNER JOIN `fileupload_samplefilegroup` ON ( `fileupload_picture`.`id` = `fileupload_samplefilegroup`.`posdata_id` ) INNER JOIN `fileupload_sample` ON ( `fileupload_samplefilegroup`.`id` = `fileupload_sample`.`samplefile_id` ) WHERE (`fileupload_sample`.`id`) IN (SELECT DISTINCT U0.`id` FROM `fileupload_sample` U0 INNER JOIN `groups_sampleattribute` U1 ON ( U0.`id` = U1.`sample_id` ) INNER JOIN `groups_attribute` U2 ON ( U1.`attribute_id` = U2.`id` ) INNER JOIN `experiments_attributecomparison` U3 ON ( U2.`id` = U3.`attribute_id` ) INNER JOIN `experiments_comparison` U4 ON ( U3.`comparison_id` = U4.`id` ) WHERE U4.`experiment_id` = 1); args=(1,)
DEBUG 2016-09-26 11:48:25,245 utils.py:91 | (0.001) SELECT `fileupload_picture`.`file` FROM `fileupload_picture` INNER JOIN `fileupload_samplefilegroup` ON ( `fileupload_picture`.`id` = `fileupload_samplefilegroup`.`negdata_id` ) INN

 "PiMP log level set to DEBUG"
2016-09-26 11:48:24 ERROR:Pimp.extra.initialisePiMPLogging:Logging at ERROR level
2016-09-26 11:48:24 WARNING:Pimp.extra.initialisePiMPLogging:Logging at WARNING level
2016-09-26 11:48:24 INFO:Pimp.extra.initialisePiMPLogging:Logging at INFO level
2016-09-26 11:48:24 DEBUG:Pimp.extra.initialisePiMPLogging:Logging at DEBUG level
2016-09-26 11:48:25 INFO:PeakML.Isotope.plotSamples:OS type: x86_64-apple-darwin15.5.0
2016-09-26 11:48:25 INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx8192m
2016-09-26 11:48:25 INFO:PeakML.Isotope.plotSamples:Using jar file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2016-09-26 11:48:25 INFO:PeakML.Isotope.plotSamples:Using SWT file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/swt-4.5-cocoa-macosx-x86_64.jar


DEBUG 2016-09-26 11:48:25,308 utils.py:91 | (0.005) SELECT `experiments_database`.`name` FROM `experiments_database` INNER JOIN `experiments_params_databases` ON ( `experiments_database`.`id` = `experiments_params_databases`.`database_id` ) INNER JOIN `experiments_params` ON ( `experiments_params_databases`.`params_id` = `experiments_params`.`id` ) INNER JOIN `experiments_analysis` ON ( `experiments_params`.`id` = `experiments_analysis`.`params_id` ) WHERE `experiments_analysis`.`id` = 1; args=(1,)


### Input validations

validate that all the input files exist

In [38]:
pos_files = pipeline.metadata.files['positive']
neg_files = pipeline.metadata.files['negative']
all_files = list(pos_files)
all_files.extend(neg_files)

import os
print all_files
print pipeline.working_dir

for f_name in all_files:
    f_path = os.path.join(pipeline.working_dir, f_name)
    print f_path, os.path.isfile(f_path)
    assert os.path.isfile(f_path)

[u'samples/POS/Beer_1_full1.mzXML', u'samples/POS/Beer_1_full2.mzXML', u'samples/POS/Beer_1_full3.mzXML', u'samples/POS/Beer_2_full1.mzXML', u'samples/POS/Beer_2_full2.mzXML', u'samples/POS/Beer_2_full3.mzXML', u'samples/POS/Beer_3_full1.mzXML', u'samples/POS/Beer_3_full2.mzXML', u'samples/POS/Beer_3_full3.mzXML', u'samples/POS/Beer_4_full1.mzXML', u'samples/POS/Beer_4_full2.mzXML', u'samples/POS/Beer_4_full3.mzXML', u'samples/NEG/Beer_1_full1.mzXML', u'samples/NEG/Beer_1_full2.mzXML', u'samples/NEG/Beer_1_full3.mzXML', u'samples/NEG/Beer_2_full1.mzXML', u'samples/NEG/Beer_2_full2.mzXML', u'samples/NEG/Beer_2_full3.mzXML', u'samples/NEG/Beer_3_full1.mzXML', u'samples/NEG/Beer_3_full2.mzXML', u'samples/NEG/Beer_3_full3.mzXML', u'samples/NEG/Beer_4_full1.mzXML', u'samples/NEG/Beer_4_full2.mzXML', u'samples/NEG/Beer_4_full3.mzXML']
/Users/joewandy/git/pimp/django_projects/pimp_data/projects/1
/Users/joewandy/git/pimp/django_projects/pimp_data/projects/1/samples/POS/Beer_1_full1.mzXML True

validate that pos and neg files are of the same length and are consistent with each other

In [43]:
assert len(pos_files) == len(neg_files)
for f in range(len(pos_files)):
    pos = os.path.basename(pos_files[f])
    neg = os.path.basename(neg_files[f])
    assert pos == neg

Beer_1_full1.mzXML Beer_1_full1.mzXML
Beer_1_full2.mzXML Beer_1_full2.mzXML
Beer_1_full3.mzXML Beer_1_full3.mzXML
Beer_2_full1.mzXML Beer_2_full1.mzXML
Beer_2_full2.mzXML Beer_2_full2.mzXML
Beer_2_full3.mzXML Beer_2_full3.mzXML
Beer_3_full1.mzXML Beer_3_full1.mzXML
Beer_3_full2.mzXML Beer_3_full2.mzXML
Beer_3_full3.mzXML Beer_3_full3.mzXML
Beer_4_full1.mzXML Beer_4_full1.mzXML
Beer_4_full2.mzXML Beer_4_full2.mzXML
Beer_4_full3.mzXML Beer_4_full3.mzXML


In [57]:
unique_files = set()
for f in all_files:
    basename = os.path.basename(f)
    front, ext = os.path.splitext(basename)
    unique_files.add(front)
    
for factor in pipeline.metadata.groups:
    for level in factor.levels:
        files = factor.level_files[level]
        for f in files:
            assert f in unique_files

[beer_colour with 2 levels, beer_taste with 2 levels]
['colour_dark', 'colour_light']
['taste_awful', 'taste_delicious']


### Run the rest of the pipeline

In [16]:
pipeline.setup()

INFO 2016-09-26 11:53:46,379 pipeline_rpy2.py:95 | Data dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects
INFO 2016-09-26 11:53:46,380 pipeline_rpy2.py:96 | Project dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1


2016-09-26 11:53:46 1:INFO:Pimp.getPimpWd:Data dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects
2016-09-26 11:53:46 1:INFO:Pimp.getPimpWd:Project dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1
before is [1] "/Users/joewandy/git/pimp/django_projects/pimp_data/projects/1"

now is /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1
2016-09-26 11:53:46 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 1
2016-09-26 11:53:46 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-26 11:53:46 1:INFO:Pimp.validateInput:Input validated correctly!
2016-09-26 11:53:46 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 1
2016-09-26 11:53:46 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-26 11:53:46 1:INFO:Pimp.getAnalysisParams:Setting params
2016-09-26 11:53:46 1:DEBUG:Pimp.getAnalysisParams:Name: iqr Value: 0.5
2016-09-26 11:53:46 1:DEBUG:Pimp.getAnalysisParams:Name: rsd Va

In [17]:
xcms_params = pipeline.get_value(pipeline.pimp_params, 'xcms.params')            
mzmatch_params = pipeline.get_value(pipeline.pimp_params, 'mzmatch.params')
peakml_params = pipeline.get_value(pipeline.pimp_params, 'peakml.params')
mzmatch_outputs = pipeline.get_value(pipeline.pimp_params, 'mzmatch.outputs')
mzmatch_filters = pipeline.get_value(pipeline.pimp_params, 'mzmatch.filters')
n_slaves = multiprocessing.cpu_count()

In [30]:
raw_data_dict = {}
groups_dict = {}
polarity_pos, polarity_neg = pipeline.metadata.files.keys()

In [33]:
print pipeline.metadata.get_files()

DEBUG 2016-09-22 13:59:56,704 utils.py:91 | (0.001) SELECT `fileupload_picture`.`file` FROM `fileupload_picture` INNER JOIN `fileupload_samplefilegroup` ON ( `fileupload_picture`.`id` = `fileupload_samplefilegroup`.`posdata_id` ) INNER JOIN `fileupload_sample` ON ( `fileupload_samplefilegroup`.`id` = `fileupload_sample`.`samplefile_id` ) WHERE (`fileupload_sample`.`id`) IN (SELECT DISTINCT U0.`id` FROM `fileupload_sample` U0 INNER JOIN `groups_sampleattribute` U1 ON ( U0.`id` = U1.`sample_id` ) INNER JOIN `groups_attribute` U2 ON ( U1.`attribute_id` = U2.`id` ) INNER JOIN `experiments_attributecomparison` U3 ON ( U2.`id` = U3.`attribute_id` ) INNER JOIN `experiments_comparison` U4 ON ( U3.`comparison_id` = U4.`id` ) WHERE U4.`experiment_id` = 4); args=(4,)
DEBUG 2016-09-22 13:59:56,707 utils.py:91 | (0.001) SELECT `fileupload_picture`.`file` FROM `fileupload_picture` INNER JOIN `fileupload_samplefilegroup` ON ( `fileupload_picture`.`id` = `fileupload_samplefilegroup`.`negdata_id` ) INN

{'positive': [u'samples/POS/Beer_1_full1.mzXML', u'samples/POS/Beer_1_full2.mzXML', u'samples/POS/Beer_1_full3.mzXML', u'samples/POS/Beer_2_full1.mzXML', u'samples/POS/Beer_2_full2.mzXML', u'samples/POS/Beer_2_full3.mzXML', u'samples/POS/Beer_3_full1.mzXML', u'samples/POS/Beer_3_full2.mzXML', u'samples/POS/Beer_3_full3.mzXML', u'samples/POS/Beer_4_full1.mzXML', u'samples/POS/Beer_4_full2.mzXML', u'samples/POS/Beer_4_full3.mzXML'], 'negative': [u'samples/NEG/Beer_1_full1.mzXML', u'samples/NEG/Beer_1_full2.mzXML', u'samples/NEG/Beer_1_full3.mzXML', u'samples/NEG/Beer_2_full1.mzXML', u'samples/NEG/Beer_2_full2.mzXML', u'samples/NEG/Beer_2_full3.mzXML', u'samples/NEG/Beer_3_full1.mzXML', u'samples/NEG/Beer_3_full2.mzXML', u'samples/NEG/Beer_3_full3.mzXML', u'samples/NEG/Beer_4_full1.mzXML', u'samples/NEG/Beer_4_full2.mzXML', u'samples/NEG/Beer_4_full3.mzXML']}


In [37]:
print pipeline.metadata.get_groups()
print pipeline.metadata.get_standards()
print pipeline.metadata.get_comparisons()

DEBUG 2016-09-22 14:00:51,823 utils.py:91 | (0.001) SELECT `groups_group`.`name`, `groups_attribute`.`name`, `fileupload_sample`.`name` FROM `groups_group` INNER JOIN `groups_attribute` ON ( `groups_group`.`id` = `groups_attribute`.`group_id` ) INNER JOIN `experiments_attributecomparison` ON ( `groups_attribute`.`id` = `experiments_attributecomparison`.`attribute_id` ) INNER JOIN `experiments_comparison` ON ( `experiments_attributecomparison`.`comparison_id` = `experiments_comparison`.`id` ) LEFT OUTER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) LEFT OUTER JOIN `fileupload_sample` ON ( `groups_sampleattribute`.`sample_id` = `fileupload_sample`.`id` ) WHERE `experiments_comparison`.`experiment_id` = 4; args=(4,)
DEBUG 2016-09-22 14:00:51,835 utils.py:91 | (0.001) SELECT `fileupload_projfile`.`file` FROM `fileupload_calibrationsample` INNER JOIN `groups_projfileattribute` ON ( `fileupload_calibrationsample`.`id` = `groups_projfil

[beer_colour with 2 levels, beer_taste with 3 levels]
[u'calibration_samples/standard/Std1_1_20150422_150810.csv', u'calibration_samples/standard/Std2_1_20150422_150711.csv', u'calibration_samples/standard/Std3_1_20150422_150553.csv']
               comparison       factor            level  group
0  beer_colour_comparison  beer_colour      colour_dark      0
1  beer_colour_comparison  beer_colour     colour_light      1
2   beer_taste_comparison   beer_taste      taste_awful      0
3   beer_taste_comparison   beer_taste  taste_delicious      1
4   beer_taste_comparison   beer_taste       taste_okay      1


## Process positive raw data

In [38]:
raw_data_pos, groups_pos = pipeline.process_raw_data(polarity_pos, xcms_params, mzmatch_params, 
                                         peakml_params, mzmatch_outputs, mzmatch_filters, n_slaves)

DEBUG 2016-09-22 14:01:28,306 utils.py:91 | (0.001) SELECT `groups_group`.`name`, `groups_attribute`.`name`, `fileupload_sample`.`name` FROM `groups_group` INNER JOIN `groups_attribute` ON ( `groups_group`.`id` = `groups_attribute`.`group_id` ) INNER JOIN `experiments_attributecomparison` ON ( `groups_attribute`.`id` = `experiments_attributecomparison`.`attribute_id` ) INNER JOIN `experiments_comparison` ON ( `experiments_attributecomparison`.`comparison_id` = `experiments_comparison`.`id` ) LEFT OUTER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) LEFT OUTER JOIN `fileupload_sample` ON ( `groups_sampleattribute`.`sample_id` = `fileupload_sample`.`id` ) WHERE `experiments_comparison`.`experiment_id` = 4; args=(4,)
DEBUG 2016-09-22 14:01:28,321 utils.py:91 | (0.001) SELECT `fileupload_picture`.`file` FROM `fileupload_picture` INNER JOIN `fileupload_samplefilegroup` ON ( `fileupload_picture`.`id` = `fileupload_samplefilegroup`.`posd

Creating /Users/joewandy/git/pimp/django_projects/pimp_data/projects/4/analysis_4/positive/combined
------------------------------------------------
positive /Users/joewandy/git/pimp/django_projects/pimp_data/projects/4/analysis_4/positive /Users/joewandy/git/pimp/django_projects/pimp_data/projects/4/analysis_4/positive/combined
------------------------------------------------
[beer_colour with 2 levels, beer_taste with 3 levels]
beer_colour
 - colour_dark [u'Beer_1_full1', u'Beer_1_full2', u'Beer_1_full3', u'Beer_2_full1', u'Beer_2_full2', u'Beer_2_full3']
 - colour_light [u'Beer_3_full1', u'Beer_3_full2', u'Beer_3_full3', u'Beer_4_full1', u'Beer_4_full2', u'Beer_4_full3']
beer_taste
 - taste_awful [u'Beer_4_full1', u'Beer_4_full2', u'Beer_4_full3']
 - taste_delicious [u'Beer_1_full1', u'Beer_1_full2', u'Beer_1_full3', u'Beer_2_full1', u'Beer_2_full2', u'Beer_2_full3']
 - taste_okay [u'Beer_3_full1', u'Beer_3_full2', u'Beer_3_full3']
(2, 3)
('group_0', (0, 0), 'colour_dark,taste_awful

In [39]:
raw_data_dict['positive'] = raw_data_pos
groups_dict['positive'] = groups_pos

## Process negative raw data

In [40]:
raw_data_neg, groups_neg = pipeline.process_raw_data(polarity_neg, xcms_params, mzmatch_params, 
                                         peakml_params, mzmatch_outputs, mzmatch_filters, n_slaves)

DEBUG 2016-09-22 14:03:32,214 utils.py:91 | (0.001) SELECT `groups_group`.`name`, `groups_attribute`.`name`, `fileupload_sample`.`name` FROM `groups_group` INNER JOIN `groups_attribute` ON ( `groups_group`.`id` = `groups_attribute`.`group_id` ) INNER JOIN `experiments_attributecomparison` ON ( `groups_attribute`.`id` = `experiments_attributecomparison`.`attribute_id` ) INNER JOIN `experiments_comparison` ON ( `experiments_attributecomparison`.`comparison_id` = `experiments_comparison`.`id` ) LEFT OUTER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) LEFT OUTER JOIN `fileupload_sample` ON ( `groups_sampleattribute`.`sample_id` = `fileupload_sample`.`id` ) WHERE `experiments_comparison`.`experiment_id` = 4; args=(4,)
DEBUG 2016-09-22 14:03:32,230 utils.py:91 | (0.001) SELECT `fileupload_picture`.`file` FROM `fileupload_picture` INNER JOIN `fileupload_samplefilegroup` ON ( `fileupload_picture`.`id` = `fileupload_samplefilegroup`.`posd

Creating /Users/joewandy/git/pimp/django_projects/pimp_data/projects/4/analysis_4/negative/combined
------------------------------------------------
negative /Users/joewandy/git/pimp/django_projects/pimp_data/projects/4/analysis_4/negative /Users/joewandy/git/pimp/django_projects/pimp_data/projects/4/analysis_4/negative/combined
------------------------------------------------
[beer_colour with 2 levels, beer_taste with 3 levels]
beer_colour
 - colour_dark [u'Beer_1_full1', u'Beer_1_full2', u'Beer_1_full3', u'Beer_2_full1', u'Beer_2_full2', u'Beer_2_full3']
 - colour_light [u'Beer_3_full1', u'Beer_3_full2', u'Beer_3_full3', u'Beer_4_full1', u'Beer_4_full2', u'Beer_4_full3']
beer_taste
 - taste_awful [u'Beer_4_full1', u'Beer_4_full2', u'Beer_4_full3']
 - taste_delicious [u'Beer_1_full1', u'Beer_1_full2', u'Beer_1_full3', u'Beer_2_full1', u'Beer_2_full2', u'Beer_2_full3']
 - taste_okay [u'Beer_3_full1', u'Beer_3_full2', u'Beer_3_full3']
(2, 3)
('group_0', (0, 0), 'colour_dark,taste_awful

In [41]:
raw_data_dict['negative'] = raw_data_neg
groups_dict['negative'] = groups_neg

### Run stats

In [42]:
print groups_dict['positive']

$group_3
[1] "Beer_4_full3" "Beer_4_full2" "Beer_4_full1"

$group_1
[1] "Beer_1_full2" "Beer_1_full3" "Beer_1_full1" "Beer_2_full1" "Beer_2_full3"
[6] "Beer_2_full2"

$group_5
[1] "Beer_3_full1" "Beer_3_full2" "Beer_3_full3"




In [43]:
print groups_dict['negative']

$group_3
[1] "Beer_4_full3" "Beer_4_full2" "Beer_4_full1"

$group_1
[1] "Beer_1_full2" "Beer_1_full3" "Beer_1_full1" "Beer_2_full1" "Beer_2_full3"
[6] "Beer_2_full2"

$group_5
[1] "Beer_3_full1" "Beer_3_full2" "Beer_3_full3"




In [44]:
save_fixtures = True
pipeline.run_stats(raw_data_dict, groups_dict, mzmatch_outputs, mzmatch_params, save_fixtures)

DEBUG 2016-09-22 14:04:56,501 utils.py:91 | (0.001) SELECT `groups_group`.`name`, `groups_attribute`.`name`, `fileupload_sample`.`name` FROM `groups_group` INNER JOIN `groups_attribute` ON ( `groups_group`.`id` = `groups_attribute`.`group_id` ) INNER JOIN `experiments_attributecomparison` ON ( `groups_attribute`.`id` = `experiments_attributecomparison`.`attribute_id` ) INNER JOIN `experiments_comparison` ON ( `experiments_attributecomparison`.`comparison_id` = `experiments_comparison`.`id` ) LEFT OUTER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) LEFT OUTER JOIN `fileupload_sample` ON ( `groups_sampleattribute`.`sample_id` = `fileupload_sample`.`id` ) WHERE `experiments_comparison`.`experiment_id` = 4; args=(4,)
